In [1]:
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from data_helper import *

In [40]:
def gradient2Image(epo_W1):
    for key in epo_W1.keys():
        W1 = epo_W1[key]
        for i in range(len(W1)):
            G = W1[i]
            maxg = G.max()
            ming = G.min()
            if np.isinf(ming) or np.isinf(maxg):
                G = np.ones(G.shape)*154
            else:
                dividor = (maxg - ming) if (maxg - ming) > 0 else 0.001
                G = (G - ming)/dividor*255.0
            W1[i] = G
        epo_W1[key] = (W1).astype(np.int).tolist()
    return epo_W1

def float_list2int(his):
    for key in his.keys():
        ll = his[key]
        ll = ll[np.logical_not(np.isnan(ll))]
        kk = ll[np.logical_not(np.isinf(ll))]
        kk = (kk*1e6).astype(np.int)
        his[key] = kk.tolist()
    return his

In [47]:
import json
json.encoder.FLOAT_REPR =lambda x:format(x,'.3f')
    
class MyEncoder(json.JSONEncoder):
    def encode(self, obj):
        if isinstance(obj, float):
            return format(obj, '.2f')
        return json.JSONEncoder.encode(self, obj)
    
def save2json(CNN_m, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epoch, val_acc, his, epo_it, epo_W1):
    data = {
        "CNN_mid_width": CNN_m,
        "CNN_d": CNN_d,
        "drop": drop,
        "reg": reg,
        "lr": lr,
        "CNN_out_width": CNN_out_width,
        "h_fc": h_fc,
        "epoch": epoch,
        "val_acc": val_acc,
        "his": float_list2int(his),
        "epo_it": epo_it,
        "epo_W1": gradient2Image(epo_W1),
    }
    
    
#     plt.imshow(gradient2Image(epo_W1)[7][1])
#     plt.axis('off')
#     plt.gcf().set_size_inches(5, 5)
#     plt.show()
#     if json_exist(hs, bs, lr, reg, num_epoch):
#         return
    with open( f'convJson/{CNN_mid_width}-{CNN_d}-{drop}-{reg}-{lr}-{CNN_out_width}-{h_fc}-{epoch}.json', 'w') as f:
#         json.dump(data, f, indent=1)
        json.dump(data, f,separators=(',', ':'))

CNN_mid_width, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epoch, val_acc, his, epo_it, epo_W1 = \
        get_pickle(128, 0, 0.2, 0.03, 0.002, 20, 50, 8)

save2json(CNN_mid_width, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epoch, val_acc, his, epo_it, epo_W1)

In [48]:
learning_rate = 2e-3
CNN_mid_width=[32, 64, 128]
# CNN_mid_width=[32]
CNN_out_width = 20
dropout = [0, 0.2, 0.5]
# dropout = [0.2]
hidden_fc = 50
epochs=8
regularization = [0.03, 0.02]
CNN_depth = [0, 2, 4, 6]

lr = learning_rate
h_fc = hidden_fc
for CNN_m in CNN_mid_width:
    for CNN_d in CNN_depth:
        for reg in regularization:
            for drop in dropout:
                if pickle_exist(CNN_m, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epochs):
                    print('.',end='')
                    CNN_mid_width, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epoch, val_acc, his, epo_it, epo_W1 = \
                        get_pickle(CNN_m, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epochs)
                    save2json(CNN_mid_width, CNN_d, drop, reg, lr, CNN_out_width, h_fc, epoch, val_acc, his, epo_it, epo_W1)

........................................................................